### OD-1.PROJECT. Определяем количество ботов на сайте

Проект выполнила: *Машковцева Дарья*

 ЗАДАНИЕ

1. Оценить количество ботов, которые заходят на наш сайт, и их долю от всего трафика.

2. Определить, с каких источников и в какое время суток чаще всего заходят роботы.

###### *Роботами могут быть пауки поисковых систем, которые индексируют контент блога, или же различные парсеры и агрегаторы контента. Яндекс.Метрика позволяет находить этих роботов и показывает их в отдельном параметрe ym:s:isRobot, а также считает их долю в специальной метрике ym:s:robotPercentage.*

In [54]:
import json
import requests
import pandas as pd
import numpy as np
from pprint import pprint

In [75]:
# Получаем данные

token = 'AQAAAAADybOOAAedRJ17-iGHOUPFs-lrxYXlefQ'
headers = {'Authorization': 'OAuth ' + token}
params = {'metrics': 'ym:s:robotPercentage,ym:s:visits',
          'dimensions': 'ym:s:isRobot,ym:s:hour,ym:s:lastsignTrafficSource',  
          'date1': '2020-09-01',  
          'date2': '2020-09-30',     
          'ids': 30177909,
          'accuracy':'full',
          'limit':100000}
response = requests.get('https://api-metrika.yandex.net/stat/v1/data', params=params, headers=headers)
metrika_data = response.json()

In [76]:
def getMetrikaDataInListOfDicts(metrika_data):
    list_of_dicts = []
    dimensions_list = metrika_data['query']['dimensions']
    metrics_list = metrika_data['query']['metrics']
    for data_item in metrika_data['data']:
        d = {}
        for i,dimension in enumerate(data_item['dimensions']):
            d[dimensions_list[i]] = dimension['name']
        for i,metric in enumerate(data_item['metrics']):
            d[metrics_list[i]] = metric
        list_of_dicts.append(d)
    return list_of_dicts

metrika_list_of_dicts = getMetrikaDataInListOfDicts(metrika_data)

metrika_df = pd.DataFrame(metrika_list_of_dicts)
metrika_df.columns=['isRobot','Hour','Source','robotPercentage','Visits']
display(metrika_df)

,isRobot,Hour,Source,robotPercentage,Visits
0,Robots,00:00,Direct traffic,100.0,2.0
1,Robots,00:00,Search engine traffic,100.0,17.0
2,Robots,01:00,Direct traffic,100.0,4.0
3,Robots,01:00,Search engine traffic,100.0,8.0
4,Robots,01:00,Cached page traffic,100.0,1.0
...,...,...,...,...,...
177,People,22:00,Social network traffic,0.0,3.0
178,People,23:00,Direct traffic,0.0,19.0
179,People,23:00,Link traffic,0.0,1.0
180,People,23:00,Search engine traffic,0.0,104.0


In [77]:
# Меняем формат
metrika_df['Hour'] = pd.to_datetime(metrika_df['Hour'])
metrika_df['Hour'] = metrika_df['Hour'].dt.time
metrika_df

,isRobot,Hour,Source,robotPercentage,Visits
0,Robots,00:00:00,Direct traffic,100.0,2.0
1,Robots,00:00:00,Search engine traffic,100.0,17.0
2,Robots,01:00:00,Direct traffic,100.0,4.0
3,Robots,01:00:00,Search engine traffic,100.0,8.0
4,Robots,01:00:00,Cached page traffic,100.0,1.0
...,...,...,...,...,...
177,People,22:00:00,Social network traffic,0.0,3.0
178,People,23:00:00,Direct traffic,0.0,19.0
179,People,23:00:00,Link traffic,0.0,1.0
180,People,23:00:00,Search engine traffic,0.0,104.0


In [78]:
# Количество ботов

metrika_df[metrika_df['isRobot'] == 'Robots'].shape[0]

77

In [91]:
metrika_df_part = metrika_df.copy()
metrika_df_part.groupby(by='isRobot').median()

,robotPercentage,Visits
isRobot,,
People,0.0,7.0
Robots,100.0,4.0


In [90]:
# Доля ботов от всего трафика

metrika_df[metrika_df['isRobot']=='Robots'].mean()

C:\Users\User\AppData\Local\Temp/ipykernel_10908/3117492090.py:3: FutureWarning: Dropping of nuisance columns in DataFrame reductions (with 'numeric_only=None') is deprecated; in a future version this will raise TypeError.  Select only valid columns before calling the reduction.
  metrika_df[metrika_df['isRobot']=='Robots'].mean()


robotPercentage    100.000000
Visits              12.493506
dtype: float64

In [72]:
metrika_df.groupby(by='Source').mean()

,robotPercentage,Visits
Source,,
Cached page traffic,16.666667,1.000000
Direct traffic,50.000000,15.770833
Internal traffic,30.000000,1.000000
Link traffic,44.736842,3.315789
Recommendation system traffic,0.000000,1.000000
Search engine traffic,50.000000,69.458333
Social network traffic,26.666667,2.666667
